In [55]:
import pandas as pd
import numpy as np
from configparser import ConfigParser
from tqdm import tqdm
import os
import tiktoken
from openai import OpenAI
from configparser import ConfigParser

## Conf

In [4]:
#setup openai client
config=ConfigParser()
config.read('conf/openai.ini')

client = OpenAI(api_key=config['openai']['apikey'])

## Load data

In [23]:
df=pd.read_excel('data/interim/summarization/for_summarization_mbart_2048_chunks.xlsx')
df.shape

(32886, 6)

## estimate number of tokens and take subset of data for training

In [24]:
#M tokens
df['ntoks_gpt35'].sum()/1000000

72.951149

In [25]:
df.index_pk.nunique()

11168

In [26]:
indexes2use=df.sample(n=500, random_state=1).index_pk.unique().tolist()
len(indexes2use)

473

In [27]:
df_sample=df[df.index_pk.isin(indexes2use)]
df_sample=df_sample[df_sample.ntoks_space>80]
df_sample.shape

(3824, 6)

In [28]:
df_sample.ntoks_space.describe()

count    3824.000000
mean      863.603295
std       221.378803
min        82.000000
25%       737.000000
50%       930.000000
75%      1034.000000
max      1516.000000
Name: ntoks_space, dtype: float64

In [29]:
df_sample.ntoks_gpt35.sum()/1e6

9.545186

## example usage

In [32]:
df_sample.text[150]

'Urmas Klaas: Austatud Riigikogu aseesimees! Lugupeetud kolleegid! Austatud minister Jaak Aaviksoo! Minu ülesanne täna on teile tutvustada, mida on kultuurikomisjon ülikooliseaduse, rakenduskõrgkooli seaduse ja teiste seaduste muutmise seaduse eelnõuga nr 89 esimese ja teise lugemise vahepeal teinud. Vabariigi Valitsus algatas selle eelnõu eelmise aasta 12. septembril, pärast seda valmistati eelnõu põhjalikult ette esimesele lugemisele esitamiseks. Esimene lugemine toimus täiskogus eelmise aasta 17. ja 22. novembril. Muudatusettepanekute esitamise tähtajaks, 6. detsembriks esitasid oma ettepanekud Eesti Keskerakonna fraktsioon, Sotsiaaldemokraatliku Erakonna fraktsioon ja Riigikogu liige Jevgeni Ossinovski. Lisaks tuli ettepanekuid Eesti Üliõpilaskondade Liidult, Eesti Ettevõtluskõrgkoolilt Mainor, EBS-ilt, Haridus- ja Teadusministeeriumilt, Rektorite Nõukogult ning TTÜ üliõpilasesinduselt. Kultuurikomisjon valmistas eelnõu teiseks lugemiseks ette kaheksal istungil. Kõigil neil osalesi

In [33]:
chunk=df_sample.text[150]
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Summarize this chunk: {chunk} Provide up to 3 short bullet points into the summary. Use only Estonian."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='- Kultuurikomisjon on teinud muudatusettepanekuid ülikooliseaduse ja teiste seaduste eelnõuga nr 89, sealhulgas tegevustoetuse taotlemise ja eraldamise põhimõtete muutmise osas.\n- Muudatusettepanekud on koostatud põhjalikult ja organiseeritud teemade kaupa seletuskirjas, võimaldades selgemalt jälgida ja arutada arutluskohti.\n- Oluline on märkida muudatusettepanekuid, mis puudutavad erakoolide tegevustoetuse taotlemist, stipendiumifondi eraldamise aluseid ja osakoormusega õppekava avamist vastavalt tulemuslepingule.', role='assistant', function_call=None, tool_calls=None)


In [34]:
print(completion.choices[0].message.content)

- Kultuurikomisjon on teinud muudatusettepanekuid ülikooliseaduse ja teiste seaduste eelnõuga nr 89, sealhulgas tegevustoetuse taotlemise ja eraldamise põhimõtete muutmise osas.
- Muudatusettepanekud on koostatud põhjalikult ja organiseeritud teemade kaupa seletuskirjas, võimaldades selgemalt jälgida ja arutada arutluskohti.
- Oluline on märkida muudatusettepanekuid, mis puudutavad erakoolide tegevustoetuse taotlemist, stipendiumifondi eraldamise aluseid ja osakoormusega õppekava avamist vastavalt tulemuslepingule.


## loop and generate summaries

In [35]:
df_sample

,text,index_pk,ntoks,ntoks_space,ntoks_mbart,ntoks_gpt35
150,Urmas Klaas: Austatud Riigikogu aseesimees! Lu...,PKP-26646,1478.0,712,1361,2352
151,\n Urmas Klaas: Ettepanekus nr 29 öeldakse sõ...,PKP-26646,1478.0,856,1602,2599
152,"\n \n Urmas Klaas: Kaks minutit jah, aitäh! ...",PKP-26646,1866.0,973,1842,2996
153,\n Urmas Klaas: Aitäh! Lugupeetud kolleeg! Ei...,PKP-26646,1941.0,1041,1905,2999
154,"\n Urmas Klaas: Tänan, kolleeg Vakra! Kultuur...",PKP-26646,1896.0,955,1860,3035
...,...,...,...,...,...,...
32874,\n Aseesimees Hanno Pevkur: Meie kuues päeva...,PKP-469664,1827.0,963,1821,3050
32875,\n Yoko Alender: Aitäh küsimuse eest! No see...,PKP-469664,1959.0,1084,1933,3071
32876,\n Aseesimees Martin Helme: Jaa. Esimene lug...,PKP-469664,1806.0,938,1776,2793
32877,"\n Andres Metsoja: Aitäh, lugupeetud istungi...",PKP-469664,1348.0,758,1342,2054


In [97]:
def get_repsonse(prompt, text):
    completion = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
          ]
        )
    
    text=completion.choices[0].message.content
    ntoks=completion.usage.total_tokens
    return text, ntoks

In [101]:
savepath='data/interim/summarization/for_summarization_mbart_2048_chunks_summaries.xlsx'

def generate_prompt(text):
    prompt=f"""Summarize text. Provide up to 3 short bullet points into the content summary what was said or asked in the text by whom. 
Use only Estonian. Mark bullets with - . Text: {text}"""
    return prompt

df_sample['summary']=''
df_sample['ntoks_used_gpt35']=-1

In [102]:
df_sample=df_sample.reset_index(drop=True)

In [107]:
n_fails=0

start_i=28
i=start_i
for text in tqdm(df_sample.text.tolist()[start_i:]):
    try:
        summary, ntoks=get_repsonse(generate_prompt(text), text)
    except Exception as e:
        print(i)
        print(e)
        summary=''
        ntoks=-1
        n_fails+=1
        if n_fails>5:
            break
    finally:
        df_sample.loc[i, 'summary']=summary
        df_sample.loc[i, 'ntoks_used_gpt35']=ntoks
        i+=1
    if i%10==0:
        df_sample.to_excel(savepath, index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3796/3796 [4:07:30<00:00,  3.91s/it]


In [113]:
df_sample.head(30)

,text,index_pk,ntoks,ntoks_space,ntoks_mbart,ntoks_gpt35,summary,ntoks_used_gpt35
0,Urmas Klaas: Austatud Riigikogu aseesimees! Lu...,PKP-26646,1478.0,712,1361,2352,- Urmas Klaas tutvustas kultuurikomisjoni tege...,2559
1,\n Urmas Klaas: Ettepanekus nr 29 öeldakse sõ...,PKP-26646,1478.0,856,1602,2599,"- Urmas Klaas selgitab ettepanekut nr 29, mis ...",2810
2,"\n \n Urmas Klaas: Kaks minutit jah, aitäh! ...",PKP-26646,1866.0,973,1842,2996,"- Urmas Klaas tutvustab seaduseelnõud, mis reg...",3208
3,\n Urmas Klaas: Aitäh! Lugupeetud kolleeg! Ei...,PKP-26646,1941.0,1041,1905,2999,- Urmas Klaas selgitab komisjonis ettevalmista...,3213
4,"\n Urmas Klaas: Tänan, kolleeg Vakra! Kultuur...",PKP-26646,1896.0,955,1860,3035,"- Urmas Klaas rääkis, kuidas kultuurikomisjoni...",3269
5,"\n Urmas Klaas: Komisjon lähtus sellest, et ü...",PKP-26646,1997.0,1056,1965,3077,"- Urmas Klaas rõhutas, et akadeemiline puhkus ...",3359
6,\n Urmas Klaas: Aitäh! Kindlasti on komisjoni...,PKP-26646,1271.0,660,1245,1997,"- Urmas Klaas rõhutas, et kõrgkoolidevaheline ...",2202
7,\n Haridus- ja teadusminister Jaak Aaviksoo: ...,PKP-26646,1985.0,1052,1979,3157,- Haridus- ja teadusminister Jaak Aaviksoo tut...,3368
8,\n Haridus- ja teadusminister Jaak Aaviksoo: ...,PKP-26646,1894.0,1022,1862,2987,- Haridus- ja teadusminister Jaak Aaviksoo: Ko...,3192
9,\n Haridus- ja teadusminister Jaak Aaviksoo: ...,PKP-26646,1927.0,967,1901,3022,- Haridus- ja teadusminister Jaak Aaviksoo sel...,3297


In [108]:
df_sample.tail(30)

,text,index_pk,ntoks,ntoks_space,ntoks_mbart,ntoks_gpt35,summary,ntoks_used_gpt35
3794,\n Marko Mihkelson: Aitäh! Eks te muidugi la...,PKP-637220,1536.0,824,1522,2384,- Marko Mihkelson räägib Eesti-Vene suhete kee...,2577
3795,\n Mihhail Lotman: Ikka veel pakutakse erine...,PKP-637220,1521.0,811,1515,2355,"- Mihhail Lotman hoiatab, et erinevad garantii...",2578
3796,\n Ruuben Kaalep: Lugupeetud juhataja! Hea v...,PKP-637220,1815.0,874,1813,2819,"- Ruuben Kaalep rõhutas, et Ukrainas otsustata...",3008
3797,\n Eerik-Niiles Kross: Hea juhataja! Head ko...,PKP-637220,1771.0,947,1753,2746,- Eerik-Niiles Kross räägib välispoliitilisest...,2930
3798,\n Indrek Saar: Lugupeetud välisminister! Au...,PKP-637220,1567.0,875,1553,2383,- Indrek Saar kiidab Eesti diplomaate ja julge...,2526
3799,\n Oudekki Loone: Head kolleegid! Kujutleme ...,PKP-637220,1736.0,922,1734,2621,- Oudekki Loone kõne Riigikogule; ajamasina ka...,2803
3800,\n Peeter Ernits: Hea juhataja! Head kolleeg...,PKP-637220,1454.0,795,1444,2251,- Peeter Ernits väljendas muret Eesti välispol...,2459
3801,"\n Siim Kallas: Mulle tundub, et kõigepealt ...",PKP-637220,1093.0,595,1091,1730,"- Siim Kallas ütles kommentaariks, et Venemaa ...",1883
3802,\n Paul Puustusmaa: Aitäh! Head inimesed! Ka...,PKP-637220,1361.0,794,1359,2140,"- Paul Puustusmaa rõõmustab, et Vene väed liig...",2381
3803,"Priit Sibul: Aitäh, austatud istungi juhataja...",PKP-642619,1544.0,848,1538,2484,- Priit Sibul tutvustas perehüvitiste seaduse ...,2654


In [112]:
!dir/s data\interim\summarization\for_summarization_mbart_2048_chunks_summaries.xlsx

 Volume in drive C is OS
 Volume Serial Number is E48A-72F6

 Directory of C:\Users\RistoHinno\python\riigikogu_teemad\riigikogu_teemad\data\interim\summarization

04/19/2024  01:11 AM         8,957,997 for_summarization_mbart_2048_chunks_summaries.xlsx
               1 File(s)      8,957,997 bytes

     Total Files Listed:
               1 File(s)      8,957,997 bytes
               0 Dir(s)  26,001,764,352 bytes free
